# 导入相关包

In [46]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime,timedelta
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")

from tgrocery import Grocery
import jieba

# PDF解析原始数据 
## 加载数据并采用pdfplumber抽取PDF中的文字和表格


In [2]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('../datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = '../datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  '../datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [3]:
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)
val_df = train_df[:1800]
train_df = train_df[1800:]

# 数据处理
## 抽取整体数据（一个sampleid内此字段内容都相同）
## 公告时间，实际购买公司

In [14]:
# 提取公司
# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
# result_matrix
train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

# train_data['text_2'] = train_lstm_input['text'].astype(str)

# train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = "理财产品"


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="发行方"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)

train_data2=train_lstm_input[train_lstm_input["实际购买公司名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["实际购买公司名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="购买公司"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期","资金来源","实际购买公司和上市公司关系"]

# train_lstm_input[train_lstm_input["认购金额(万元)"].astype(str)!="否"]

for item in other_columns_list:
    print(item)

    train_data2=train_lstm_input[train_lstm_input[item].astype(str)!="否"].reset_index(drop=True)

    # train_data2["文本类别"]=item

    tmp['text_1']=train_data2[item].astype(str)

    # tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']="其它"

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data

train_src=[]
for text,label in train_data[["text_1","label_1"]].values:
    train_src.append([label,text])


grocery_word_selector=Grocery("wordSelector")


grocery_word_selector.train(train_src)

grocery_word_selector.save()



认购金额(万元)
认购日期
资金来源
实际购买公司和上市公司关系


,text_1,label_1
0,中银保本理财-人民币按期开放理财产品,理财产品
1,中银保本理财-人民币按期开放理财产品,理财产品
2,与利率挂钩的结构性产品,理财产品
3,广发银行“薪加薪”16号XJXCKJ2578,理财产品
4,兴业银行“金雪球-优悦”保本开放式人民币理财产品(2M),理财产品
...,...,...
185903,控股参股公司,其它
185904,控股参股公司,其它
185905,控股参股公司,其它
185906,控股参股公司,其它


In [62]:


def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
    for item in title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            # text_end_iter_list.append(i.span(0)[1]) #把标题纳入text
            text_end_iter_list.append(i.span(0)[0])
    
    # for item in title_list:
    for item in s_title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(2)
            # text_end_iter_list.append(i.span(0)[1]) #把标题纳入text
            text_end_iter_list.append(i.span(0)[0])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\^","")
    count=re.findall(r".*?\^.*?",text)
    if (count is not None and len(count)>0):
        print(len(count))
        print(sample_id)
        print("————————————————")
    text=text.replace(r"\n","^").replace("（","(").replace("）",")")
    title_df=get_title(text)
    count=re.findall(r".*?\^.*?",text)
    if (count is not None and len(count)>0):
        print(len(count))
        print(sample_id)
        print("————————————————")
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    index=0
    neg_index=[]
    for title_des in title_1_df[0].values:
        for item in title_neg_words:
            if re.search(item,title_des) is not None:
                neg_index.append(index)
                break
        index+=1


    return title_1_df.drop(neg_index)
    # print(title_list)

def get_judge_title_result(val_df):

    judge_title_result=None


    for sample_id,text in tqdm(val_df[["sample_id","text"]].values):
        # print(sample_id)
        # print(text)
        judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])
    
    return judge_title_result

title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","到期","截至","意见","十二个月内","公告前","报备文件","前期"]

val_judge_title_result=get_judge_title_result(val_df.head(20))
# test_judge_title_result=get_judge_title_result(test_df)
# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)


10%|█         | 2/20 [00:00<00:02,  7.25it/s]125
1739
————————————————
179
1806
————————————————
23
7032
————————————————
 20%|██        | 4/20 [00:00<00:02,  7.44it/s]208
2974
————————————————
98
6903
————————————————
 40%|████      | 8/20 [00:04<00:06,  1.99it/s]902
446
————————————————
67
9083
————————————————
85
3581
————————————————
 45%|████▌     | 9/20 [00:04<00:04,  2.21it/s]374
4188
————————————————
76
7237
————————————————
118
2765
————————————————
 65%|██████▌   | 13/20 [00:05<00:01,  3.73it/s]91
4791
————————————————
112
9075
————————————————
 70%|███████   | 14/20 [00:05<00:01,  3.74it/s]183
9004
————————————————
128
7617
————————————————
 80%|████████  | 16/20 [00:05<00:00,  4.34it/s]169
4028
————————————————
131
7618
————————————————
 90%|█████████ | 18/20 [00:06<00:00,  5.09it/s]143
2996
————————————————
53
6220
————————————————
100%|██████████| 20/20 [00:06<00:00,  3.00it/s]262
3247
————————————————



In [6]:
import jieba.analyse
import jieba

text_list=[]
for product_name in tqdm(train_outputs["产品发行方名称"].values):
    text_list.append(str(product_name))

# text_list

a_list=[]

for x in jieba.analyse.extract_tags((",").join(i for i in text_list),topK=100):#可以再添加一个参数指定输出个数
    a_list.append(x)#直接输出关键词和词频

# a_list

# text_list=[]
# for product_name in tqdm(train_outputs["理财产品名称"].values):
#     text_list.append(str(product_name))

# b_list=[]

# for x in jieba.analyse.extract_tags((",").join(i for i in text_list)):#可以再添加一个参数指定输出个数
#     b_list.append(x)

# set(b_list).difference(a_list)

100%|██████████| 32818/32818 [00:00<00:00, 301057.63it/s]
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lsqlh\AppData\Local\Temp\jieba.cache
Loading model cost 2.130 seconds.
Prefix dict has been built successfully.


In [53]:
a="中国银行股份[有限公司]辽宁省分行"
grocery_word_selector.predict(a).dec_values

a="信赢步步高升4号(B15C0873)"
a=jieba.cut(a)
print(list(a))

{'理财产品': -0.32860552275738786,
 '发行方': 1.0606784357993329,
 '购买公司': -0.42195233996337866,
 '其它': -0.3101205730785791}

['信赢', '步步高升', '4', '号', '(', 'B15C0873', ')']


In [90]:
# firm=['公司', '银行', '分行', '支行', '中心', '业部', '商行', '建行']
sample_id=7618

text_list=[]
for item in val_judge_title_result[val_judge_title_result["sample_id"]==sample_id][4].values:
    text_list.append(item)

text=("").join(i for i in text_list)
text

firm=['公司']
for word in firm:
    char_list=[]
    for char in word:
        char_list.append("["+char+"]")
    word=("").join(i for i in char_list)
    firm_pattern=re.compile("[0-9A-Za-z\u4e00-\u9fa5\^ ]+?"+word+"[0-9A-Za-z\u4e00-\u9fa5\^ ]*?")
    re_result=firm_pattern.findall(text)

    if re_result is None:
        break

    tmp_score=grocery_word_selector.predict(item).dec_values
    result={}

    for key in tmp_score:
        result[key]=[]
    
    for item in re_result:
        w_s_result=grocery_word_selector.predict(item)
        key=str(w_s_result)
        if(w_s_result.dec_values[key]>0.5):
            result[key].append(item)


"['证券代码：002782         证券简称：可立克         公告编号：2016-081 ^ ^深圳可立克科技股份有限公司 ^关于使用闲置自有资金购买保本型银行理财产品的^实施公告 ^公司及董事会全体成员保证公告内容的真实、准确和完整，对公告的虚假^记载、误导性陈述或者重大遗漏负连带责任。 ^深圳可立克科技股份有限公司（以下简称“公司”或“本公司”）于 2016^年2月 1日在公司会议室召开了第二届董事会第十三次会议，审议通过了《关于^公司使用部分闲置自有资金购买理财产品的议案》，为提高资金使用效率，在确^保公司日常运营和资金安全的情况下，同意公司使用最高不超过人民币 16,000^万元的自有资金进行现金管理，用于购买低风险、短期的、由商业银行发行的保^本型理财产品。购买理财产品的额度在董事会审议通过之日起 12 个月有效期内^可以滚动使用，并授权董事长行使该项投资决策权、由财务负责人负责具体购买^事宜。公司独立董事、监事会、保荐机构已分别对此发表了同意的意见。具体内^容详见公司《深圳可立克科技股份有限公司关于使用部分闲置自有资金购买理财^产品的公告》（临时公告 2016-012号）。  ^一、本次使用部分闲置自有资金购买理财产品的实施情况  ^1、产品名称：招商银行点金公司理财之人民币岁月流金 51442号理财计划。  ^2、产品类型：保本浮动收益型。  ^3、理财产品投资方向：本理财计划由招商银行投资于我国银行间市场信用^级别较高、流动性较好的金融资产，包括但不限于国债、金融债、央行票据、债^券回购、资金拆借、银行存款以及高信用级别的企业债、公司债、短期融资券、^中期票据、资产支付证券、次级债等其他金融资产，并可投资于可转换债券、可^分离债、新股申购、交易所债券等其他金融资产。 ^4、预期最高收益率：年化收益率为2.75%。  ^5、产品期限：2016年10月18日—2017年1月16日。  ^1 ^ ', '6、认购金额：人民币 10,000,000.00元。  ^7、资金来源：公司闲置自有资金。  ^8、公司与招商银行股份有限公司深圳新时代支行无关联关系。  ^"

{'理财产品': [' ^关于使用闲置自有资金购买保本型银行理财产品的^实施公告 ^公司'], '发行方': ['招商银行点金公司', '公司与招商银行股份有限公司'], '购买公司': ['081 ^ ^深圳可立克科技股份有限公司', ' ^深圳可立克科技股份有限公司', '深圳可立克科技股份有限公司'], '其它': []}


#### 1.抽取公告时间

In [ ]:
# 首先针对任务抽取时间（每个时间跟每个id是一一对应的）
# 要不是取第一个时间，要不就是取最后一个时间（或者时间加一）这里可以建立一个模型预测
# base这里面直接取最后一个时间作为发布日期

CN_NUM = {
    u'〇': 0, u'一': 1, u'二': 2, u'三': 3,
    u'四': 4, u'五': 5, u'六': 6, u'七': 7,
    u'八': 8, u'九': 9, u'零': 0, u'壹': 1,
    u'贰': 2, u'叁': 3, u'肆': 4, u'伍': 5,
    u'陆': 6, u'柒': 7, u'捌': 8, u'玖': 9,
    u'貮': 2, u'两': 2,
}


def get_put_time_from_text(row):
    row = row.replace(' ', '').replace('\\n', '')
    for key in CN_NUM:
        row = row.replace(key, str(CN_NUM[key]))   
    r = row.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").strip()
    regex = "(\d{4}-\d{1,2}-\d{1,2})"
    r = re.findall(regex, r)
    if len(r)==0:
        return np.nan
    time_str = r[-1]
    first = time_str.split('-')[0]
    second = time_str.split('-')[1]
    last = time_str.split('-')[-1]
    second = str.zfill(second, 2)
    last = str.zfill(last, 2)
    r = '-'.join([first, second, last])
    return r

val_result = pd.DataFrame()
val_result['sample_id'] = val_df['sample_id']
val_result['predict_time'] = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_gg = train_outputs.groupby('sample_id').apply(lambda row:list(row['公告日期'])[0]).reset_index()
test_gg.columns = ['sample_id', 'time']
val_result = pd.merge(val_result, test_gg, on='sample_id', how='left')

# 判断验证集的准确率
np.sum(val_result['predict_time'].astype(str) == val_result['time'].astype(str))/len(val_result)

val_time = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
# test_time = test_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)

#### 2.抽取实际购买公司

In [ ]:
# 抽取购买公司
# 前几句话出现
# 将其按照\\n 和空格切割
def get_gm(row):
    result = re.split('[\\\\n ]',row)
    for i in result:
        if '公司' in i:
            return i

val_gm = val_df.progress_apply(lambda row:get_gm(row['text']), axis=1)
# test_gm = test_df.progress_apply(lambda row:get_gm(row['text']), axis=1)

In [ ]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段

# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
result_matrix
train_lstm_input = pd.merge(train_table_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

train_data['text_2'] = train_lstm_input['text'].astype(str)

train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = label_1.fit_transform(train_lstm_input["文本类别"])


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="无"].reset_index(drop=True)

train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']=label_1.fit_transform(train_data2["文本类别"])

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期"]

for item in other_columns_list:

    train_data2=train_lstm_input[train_lstm_input[item]!="无"].reset_index(drop=True)

    train_data2["文本类别"]="其他"

    tmp['text_1']=train_data2[item].astype(str)

    tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']=label_1.fit_transform(train_data2["文本类别"])

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data
